# Sample analysis

In [ ]:
from BlindersPy3 import Blinders, FitType
from precessionlib.util import *
from precessionlib.fitmodels import *
from precessionlib.analysis import *

In [ ]:
import json 
with open('run2CHannahTest.json') as file:
        config = json.load(file)

config['include_y_osc'] = False

# config['extended_fit_end'] = 200

config['do_start_time_scans'] = False
        
print(config['file_name'])
print(config['hist_name'])

In [ ]:
master_3d = get_histogram(config['hist_name'], config['file_name'])
calo_range = config['calo_range']
master_3d.GetZaxis().SetRange(calo_range[0], calo_range[1])

In [ ]:
r.gStyle.SetStatW(0.25)
r.gStyle.SetStatH(0.4)

## Start With the Basic All Calo T-Method Analysis

In [ ]:
all_calo_2d = master_3d.Project3D('yx_all')

In [ ]:
blinder = Blinders(FitType.Omega_a, config['blinding_phrase'])
omega_a_ref = blinder.paramToFreq(0)

five_param_tf1 = build_5_param_func(config)
five_param_tf1.SetParameters(0, 64.4, 0.2, 0.2, 0, omega_a_ref)
five_param_tf1.FixParameter(5, omega_a_ref)

Sweep over energy threshold to find the optimal T-Method cut

In [ ]:
optimal_thresh_bin, sweep_res = do_threshold_sweep(all_calo_2d, five_param_tf1, 
                                                   config['fit_start'], 
                                                   config['extended_fit_end'])
if config['fix_thresh_bin']:
        optimal_thresh_bin = config['thresh_bin']
        print(f'forcing thresh bin to {optimal_thresh_bin}')

In [ ]:
c, _ = plot_threshold_sweep(all_calo_2d, sweep_res, optimal_thresh_bin)

In [ ]:
c.Draw()

## Five-parameter fit

In [ ]:
# fix thresh bin if configured to do so
if config['fix_thresh_bin']:
    optimal_thresh_bin = config['thresh_bin']

In [ ]:
best_T_hist = all_calo_2d.ProjectionX('optimal T', optimal_thresh_bin, -1)

In [ ]:
five_param_tf1.SetParameter(0,best_T_hist.GetBinContent(
                                    best_T_hist.FindBin(30)) * 1.6)
bin_width = best_T_hist.GetBinWidth(1)

best_thresh = all_calo_2d.GetYaxis().GetBinCenter(optimal_thresh_bin)
best_T_hist.SetTitle(
        f'T-Method, all calos, {best_thresh/1000:.2f} GeV threshold; ' +
        f'time [#mus]; N / {bin_width:.3f} #mus')

In [ ]:
resids, fft, cbo_freq = fit_and_fft(
        best_T_hist, five_param_tf1, 'fiveParamAllCalos',
        config['fit_options'],
        config['fit_start'], config['extended_fit_end'], True)

In [ ]:
c, _ = plot_hist(best_T_hist)
c.Draw()

c2, _ = plot_fft(fft, cbo_freq)
c2.Draw()

In [ ]:
c, _ = make_wrapped_wiggle_plot(best_T_hist, five_param_tf1, '5-Parameter Fit', 
                               config['fit_start'], config['extended_fit_end'])
r.gStyle.SetOptStat(0)
r.gStyle.SetOptFit(0)
c.Draw()

In [ ]:
early_resids = build_residuals_hist(best_T_hist, five_param_tf1, use_errors=True,
                                   name='5ParamErrorWeighted')


c = r.TCanvas()
c.Divide(1 ,2, 0, 0)
r.gStyle.SetOptTitle(0)
c.cd(1)
resid_T_hist = best_T_hist.Clone()
resid_T_hist.SetName('resid5ParamHist')
resid_T_hist.Draw()
resid_T_hist.GetXaxis().SetRangeUser(config['fit_start']-20, config['fit_start']+40)
resid_T_hist.GetXaxis().SetLabelOffset(999)

c.cd(2)
early_resids.Draw()
early_resids.SetTitle('5-Parameter Residuals; time [#mus]; pull')
early_resids.GetYaxis().SetRangeUser(-10, 10)
early_resids.GetXaxis().SetRangeUser(config['fit_start']-20, config['fit_start']+40)
early_resids.Draw()
line = r.TLine(config['fit_start'], -10, config['fit_start'], 10)
line.Draw()
line2 = r.TLine(config['fit_start']-20, 0, config['fit_start']+40, 0)
line2.Draw()
c.Draw()

In [ ]:
r.gStyle.SetOptFit(1111)
r.gStyle.SetOptTitle(1)

### Include the CBO N-term

In [ ]:
with_cbo_tf1 = build_CBO_only_func(five_param_tf1, cbo_freq, config)

resids, fft = fit_and_fft(
        best_T_hist, with_cbo_tf1, 'cboFitAllCalos',
        config['fit_options'],
        config['fit_start'], config['fit_end'])

In [ ]:
c, _ = plot_hist(best_T_hist)
c.Draw()

c2, _ = plot_fft(fft, cbo_freq)
c2.Draw()

### Include the vertical waist 

In [ ]:
cbo_freq = with_cbo_tf1.GetParameter(9) / 2 / math.pi

vw_tf1 = build_CBO_VW_func(with_cbo_tf1, cbo_freq, config)
resids, fft = fit_and_fft(best_T_hist, vw_tf1, 'vwFitAllCalos',
              config['fit_options'],
              config['fit_start'], config['fit_end'])

In [ ]:
c, _ = plot_hist(best_T_hist)
c.Draw()

c2, _ = plot_fft(fft, cbo_freq)
c2.Draw()

### Include the muon loss term

In [ ]:
muon_hists = prepare_loss_hist(config, best_T_hist)

c, _ = plot_loss_hists(*muon_hists[1:])
c.Draw()

In [ ]:
loss_tf1 = build_losses_func(vw_tf1, config)
resids, fft = fit_and_fft(
        best_T_hist, loss_tf1, 'lossFitAllCalos',
        config['fit_options'],
        config['fit_start'], config['extended_fit_end'])

In [ ]:
c, _ = plot_hist(best_T_hist)
c.Draw()

c2, _ = plot_fft(fft, cbo_freq)
c2.Draw()

### Include full fit with changing CBO and CBO modulation of A, phi
Also fit over extended range

In [ ]:
full_fit_tf1 = build_full_fit_tf1(loss_tf1, config)

resids, fft = fit_and_fft(
        best_T_hist, full_fit_tf1, 'fullFitAllCalos',
        config['fit_options'],
        config['fit_start'], config['extended_fit_end'])

In [ ]:
c, _ = plot_hist(best_T_hist)
c.Draw()
best_T_hist.GetYaxis().SetRangeUser(10, best_T_hist.GetMaximum())
c2, _ = plot_fft(fft, cbo_freq)
fft.Draw()
c2.Draw()
fft.SetTitle(';;')
# fft.GetYaxis().SetLabelOffset(999)
c2.Draw()

In [ ]:
for par_num in range(15, 19):
    print(f'[\"{full_fit_tf1.GetParName(par_num)}\", {full_fit_tf1.GetParameter(par_num)}]')

### Redo fit with pu uncertainty factors if configured to do so

In [ ]:
pu_unc_file = config.get('pu_uncertainty_file', None)

if pu_unc_file is not None:
    # load pileup uncertainty factors
    factor_array = np.loadtxt(pu_unc_file, skiprows=1)
    T_meth_unc_facs = factor_array[:, 1]
    A_weight_unc_facs = factor_array[:, 2]
else:
    T_meth_unc_facs = []
    A_weight_unc_facs = []

In [ ]:
if len(T_meth_unc_facs):
    if len(T_meth_unc_facs) != best_T_hist.GetNbinsX():
            raise ValueError('Number of T-Method pileup uncertainty factors'
                             ' does not match the number of bins'
                             ' in the T-Method histogram!')
    T_fac_hist = r.TH1D('TMethUncFacs', 'TMethUncFacs',
                best_T_hist.GetNbinsX(),
                best_T_hist.GetBinLowEdge(1), 
                best_T_hist.GetBinLowEdge(best_T_hist.GetNbinsX()+1))
    
    for i_bin in range(1, best_T_hist.GetNbinsX() + 1):
            center = best_T_hist.GetBinCenter(i_bin)
            new_err = np.sqrt(full_fit_tf1.Eval(center)) \
                * T_meth_unc_facs[i_bin - 1]

            best_T_hist.SetBinError(i_bin, new_err)
            T_fac_hist.SetBinContent(i_bin, T_meth_unc_facs[i_bin - 1])
    c_fac = r.TCanvas()
    T_fac_hist.Draw()
    T_fac_hist.GetXaxis().SetRangeUser(30,100)
    T_fac_hist.GetYaxis().SetRangeUser(1,1.01)
    c_fac.Draw()
    
    before_r = full_fit_tf1.GetParameter(4)

    c.cd()
    resids, fft = fit_and_fft(
        best_T_hist, full_fit_tf1, 'fullFitAllCalosUncFacs',
        config['fit_options'],
        config['fit_start'], config['extended_fit_end'])
    
    after_r = full_fit_tf1.GetParameter(4)
    
    c, _ = plot_hist(best_T_hist)
    c.SetLogy(1)
    c.Draw()
    c2, _ = plot_fft(fft, cbo_freq)
    c2.Draw()
    
    print(f'Before: {before_r}')
    print(f'After: {after_r}')
    print(f'Diff: {after_r - before_r}')

Do a fit saving the covariance matrix so I can print out correllations with R

In [ ]:
fr = best_T_hist.Fit(full_fit_tf1, 'EMqS', '', config['fit_start'], config['extended_fit_end'])

In [ ]:
from IPython.display import display, Markdown, Latex

In [ ]:
corr_mat = fr.GetCorrelationMatrix()
for par_num in range(full_fit_tf1.GetNpar()):
    if is_free_param(full_fit_tf1, par_num):
        par_name = full_fit_tf1.GetParName(par_num).replace('#', '\\')
        val = full_fit_tf1.GetParameter(par_num)
        err = full_fit_tf1.GetParError(par_num)
        corr =  corr_mat(4, par_num)
        display(Latex(f'${par_name}$: {val} $\pm$ {err}, R-corr {corr:.3f}'))

### Make wrapped wiggle plot and residuals plot

In [ ]:
c, _ = make_wrapped_wiggle_plot(best_T_hist, full_fit_tf1, 'T-Method', 
                               config['fit_start'], config['extended_fit_end'])
r.gStyle.SetOptStat(0)
r.gStyle.SetOptFit(0)
c.Draw()

In [ ]:
early_resids = build_residuals_hist(best_T_hist, full_fit_tf1, use_errors=True,
                                   name='fullFitErrorWeighted')

c = r.TCanvas()
# c.Divide(1 ,2, 0, 0)
c.Divide(1, 2)
r.gStyle.SetOptTitle(0)
c.cd(1)
resid_T_hist = best_T_hist.Clone()
resid_T_hist.SetName('residFullFit')
resid_T_hist.Draw()
resid_T_hist.GetXaxis().SetRangeUser(config['fit_start']-20, config['fit_start']+40)
# resid_T_hist.GetXaxis().SetLabelSize(0.1)
# resid_T_hist.GetXaxis().SetLabelOffset(999)
resid_T_hist.GetXaxis().SetLabelSize(0.1)
# resid_T_hist.GetYaxis().SetLabelOffset(999)
resid_T_hist.GetXaxis().SetTitle('')
resid_T_hist.GetYaxis().SetTitle('')
resid_T_hist.GetYaxis().SetRangeUser(0, 7e6)
resid_T_hist.GetYaxis().SetLabelSize(0.1)
resid_T_hist.GetYaxis().SetNdivisions(5, 5, 0, True)

c.cd(2)
early_resids.Draw()
early_resids.SetTitle('5-Parameter Residuals; time [#mus]; pull')
early_resids.GetYaxis().SetRangeUser(-5, 15)
early_resids.GetXaxis().SetRangeUser(config['fit_start']-20, config['fit_start']+40)
early_resids.Draw()
early_resids.GetXaxis().SetLabelSize(0.1)
early_resids.GetXaxis().SetTitle('')
early_resids.GetYaxis().SetTitle('')
early_resids.GetYaxis().SetLabelSize(0.1)
early_resids.GetYaxis().SetNdivisions(5, 5, 0, True)

line = r.TLine(config['fit_start'], -5, config['fit_start'], 15)
line.Draw()
line2 = r.TLine(config['fit_start']-20, 0, config['fit_start']+40, 0)
line2.Draw()
c.Draw()

In [ ]:
r.gStyle.SetOptFit(1111)
r.gStyle.SetOptTitle(1)
t_resid_hist = get_residuals_distribution(early_resids, 'T-Method', config)
        
c = r.TCanvas()
t_resid_hist.Draw()
c.Draw()

## Plot the Absolute Loss Numbers

In [ ]:
loss_from_zero, loss_from_start = make_loss_correction_hists(muon_hists[0], 
                                                             full_fit_tf1.GetParameter(14),
                                                             config)

In [ ]:
c = r.TCanvas()
loss_from_zero.Draw('hist')
c.Draw()

In [ ]:
c = r.TCanvas()
loss_from_start.Draw('hist')
c.Draw()

## T-Method stop time scan

In [ ]:
r.gStyle.SetOptFit(1111)
r.gStyle.SetOptTitle(1)

In [ ]:
stop_time_fit = clone_full_fit_tf1(full_fit_tf1, 'start_time_fit')
#fix a lot of parameters for the start time scan
# for par_num in start_time_conf['params_to_fix']:
for par_num in range(5, 27):
    stop_time_fit.FixParameter(par_num, 
                                stop_time_fit.GetParameter(par_num))

In [ ]:
%%time
if config['do_start_time_scans']:
    t_stop_chi2g, stop_scan_res = stop_time_scan(best_T_hist, stop_time_fit,
                                  start=config['fit_start'],
                                  end=config['extended_fit_end'],
                                  step=2, n_pts=150,
                                  fit_options=config['fit_options']+'E')

In [ ]:
if config['do_start_time_scans']:
    canvs = [r.TCanvas()]
    t_stop_chi2g.Draw('ap')
    canvs[-1].Draw()

    for i, res in enumerate(stop_scan_res):
        canvs.append(r.TCanvas())
        res.Draw()
        canvs[-1].Draw()

## T-Method start time scan

In [ ]:
start_time_fit = clone_full_fit_tf1(full_fit_tf1, 'start_time_fit')
start_time_conf = config['start_time_scan']
# fix a lot of parameters for the start time scan
# for par_num in start_time_conf['params_to_fix']:
for par_num in range(5, 27):
    start_time_fit.FixParameter(par_num, 
                                start_time_fit.GetParameter(par_num))

In [ ]:
%%time
if config['do_start_time_scans']:
    t_scan_chi2g, scan_res = start_time_scan(best_T_hist, start_time_fit,
                              start=config['fit_start'],
                              end=config['extended_fit_end'],
                              step=start_time_conf['step'], n_pts=start_time_conf['n_pts'],
                              fit_options=config['fit_options']+'E')

In [ ]:
if config['do_start_time_scans']:
    canvs = [r.TCanvas()]
    t_scan_chi2g.Draw('ap')
    canvs[-1].Draw()

    for i, res in enumerate(scan_res):
        canvs.append(r.TCanvas())
        res.Draw()
        canvs[-1].Draw()

## T-Method Pileup Multiplier Scan
Note that the optimal multipler for $\chi^2$ depends on whether I include a loss parameter, which energy bin I look at, fit start time, etc. So, it appears it will be difficult to interpret it until all the gain corrections are implemented.

In [ ]:
uncorrected_3d = get_histogram(config['uncor_hist_name'], config['file_name'])
uncorrected_3d.GetZaxis().SetRange(calo_range[0], calo_range[1])

In [ ]:
uncorrected_2d = uncorrected_3d.Project3D('yx')
uncorrected_T_hist = uncorrected_2d.ProjectionX('uncorrectedT', optimal_thresh_bin, -1)
corrected_T_hist = best_T_hist

Find some decent starting parameters

In [ ]:
pu_scan_fit = clone_full_fit_tf1(full_fit_tf1, 'pu_scan_fit')
pu_scan_fit.SetParLimits(6, 100, 400)

In [ ]:
%%time
scale_factors = [i/10 + 0.1 for i in range(15)]
pu_scan_fits = T_meth_pu_mult_scan(all_calo_2d, uncorrected_2d, 
                          optimal_thresh_bin, pu_scan_fit,
                          scale_factors, config)

In [ ]:
chi2_g, par_gs = make_pu_scan_graphs(scale_factors, pu_scan_fits)

In [ ]:
c1 = plot_pu_sweep_R(par_gs, 'T-Method')
c1.Draw()
c2 = plot_pu_sweep_chi2(chi2_g, 'T-Method')
c2.Draw()

# Per calorimeter T-Method Sweep

In [ ]:
per_calo_fit = clone_full_fit_tf1(full_fit_tf1, 'per_calo_fit')

# free 2*omega_cbo params for per calo
# for par_num in range(24, 27):
#     per_calo_fit.ReleaseParameter(par_num)
# per_calo_fit.SetParLimits(24, 30, 200)

# limit the cbo lifetime params
per_calo_fit.SetParLimits(6, 50, 400)

# fix vw parameters for the single calo fit
for par_num in [10, 13]:
    per_calo_fit.FixParameter(par_num,
                              per_calo_fit.GetParameter(par_num))

Do the sweep, it takes about five minutes

In [ ]:
%%time
calo_sweep_res = T_method_calo_sweep(
        master_3d, per_calo_fit, 
        optimal_thresh_bin, config)

Plot calo sweep results

In [ ]:
chi2_g, par_gs = make_calo_sweep_graphs(calo_sweep_res)

In [ ]:
r.gStyle.SetOptFit(0)
r.gStyle.SetOptFit(1111)
r.gStyle.SetStatH(0.15)

canvs = []
lns = []

chi2_g.SetName('calo_chi2_g')
chi2_g.SetTitle(';calo num; #chi^{2}/ndf')


for hist, fit, _, fft in calo_sweep_res:

    cbo_freq = fit.GetParameter(9) / 2 / math.pi
    
    c, _ = plot_hist(hist)
    canvs.append(c)
    r.gStyle.SetOptFit(0)
    r.gStyle.SetOptFit(1111)
    canvs[-1].Draw()

    c, new_lns = plot_fft(fft, cbo_freq)
    canvs.append(c)
    lns.extend(new_lns)
    canvs[-1].Draw()

In [ ]:
c = r.TCanvas()
chi2_g.Draw('ap')
chi2_g.GetXaxis().SetLimits(0, 25)
ln = r.TLine(0, 1, 25, 1)
ln.SetLineWidth(2)
ln.SetLineStyle(2)
ln.Draw()
c.Draw()

In [ ]:
canvs = []
for par_num in par_gs:
    canvs.append(r.TCanvas())
    g = par_gs[par_num]
    g.GetXaxis().SetTitle('calo num')

    g.Draw('ap')

    par_name = g.GetYaxis().GetTitle()
    par_name = strip_par_name(par_name)

    if par_name == 'R' or par_name=='tau':
        g.Fit('pol0')
    
    g.GetXaxis().SetLimits(0, 25)

    canvs[-1].Draw()

# All Calo Energy-Binned Analysis

In [ ]:
e_binned_fit = clone_full_fit_tf1(full_fit_tf1, 'e_binned_fit')

# fix a number of parameters for the energy binned fits
for par_num in [6, 9, 10, 13] + list(range(19, 27)):
    e_binned_fit.FixParameter(par_num, e_binned_fit.GetParameter(par_num))

# for par_num in range(15,19):
#     e_binned_fit.FixParameter(par_num, 0)

Do the energy sweep. Again, this takes a few minutes.

In [ ]:
%%time
e_sweep_res = energy_sweep(
    master_3d, e_binned_fit, config)

In [ ]:
chi2_g, par_gs = make_E_sweep_graphs(e_sweep_res)

In [ ]:
r.gStyle.SetOptFit(0)
r.gStyle.SetOptFit(1111)
r.gStyle.SetStatH(0.15)

chi2_g.SetName('energy_chi2_g')
chi2_g.SetTitle(';energy [MeV]; #chi^{2}/ndf')

canvs = []
lns = []

for pt_num, (hist, (low_e, high_e), fit, _, fft) in \
        enumerate(e_sweep_res):

    energy = 0.5 * (low_e + high_e)

    c, _ = plot_hist(hist)
    r.gStyle.SetStatH(0.15)
    canvs.append(c)
    canvs[-1].Draw()

    cbo_freq = fit.GetParameter(9) / 2 / math.pi
    c, new_lns = plot_fft(fft, cbo_freq)
    canvs.append(c)
    lns.extend(new_lns)
    canvs[-1].Draw()

In [ ]:
c = r.TCanvas()
chi2_g.Draw('ap')
c.Draw()

In [ ]:
canvs = []
for par_num in par_gs:
    canvs.append(r.TCanvas())
    g = par_gs[par_num]
    g.GetXaxis().SetTitle('energy [MeV]')

    g.Draw('ap')

    par_name = g.GetYaxis().GetTitle()
    par_name = strip_par_name(par_name)

    canvs[-1].Draw()

# save graphs we'll need for the A-Weighted analysis
a_vs_e = par_gs[2]
phi_vs_e = par_gs[3]

# A-Weighted Analysis

First create a signed version of Asymmetry versus E

In [ ]:
signed_a_vs_e, a_vs_e_spline = build_A_vs_E_spline(a_vs_e, phi_vs_e)

In [ ]:
c, _ = plot_a_vs_e_curve(signed_a_vs_e, a_vs_e_spline)

# c = r.TCanvas()
# signed_a_vs_e.Draw('ap')
# signed_a_vs_e.GetXaxis().SetLimits(0, 3000)
# a_vs_e_spline.Draw('same')
# ln = r.TLine(0, 0, 3000, 0)
# ln.SetLineStyle(2)
# ln.Draw()
c.Draw()

Now build the A-Weighted histogram

In [ ]:
a_weight_hist = build_a_weight_hist(all_calo_2d, a_vs_e_spline, 'a_weight_hist')

In [ ]:
a_weight_fit = clone_full_fit_tf1(full_fit_tf1, 'a_weight_fit')

# a_weight_fit.FixParameter(13, a_weight_fit.GetParameter(13))

a_weight_fit.SetParameter(0,
                          a_weight_hist.GetBinContent(
                              a_weight_hist.FindBin(30))*1.6)


In [ ]:
%%time
resids, fft = fit_and_fft(
        a_weight_hist, a_weight_fit, 'fullFitAWeight',
        config['fit_options'],
        config['fit_start'], config['extended_fit_end'])

In [ ]:
c, _ = plot_hist(a_weight_hist)
a_weight_hist.GetYaxis().SetRangeUser(10, a_weight_hist.GetMaximum())

c.Draw()

c2, _ = plot_fft(fft, cbo_freq)
c2.Draw()

In [ ]:
if len(A_weight_unc_facs):
    if len(A_weight_unc_facs) != a_weight_hist.GetNbinsX():
            raise ValueError('Number of A-Weighted pileup uncertainty factors'
                             ' does not match the number of bins'
                             ' in the A-Weighted histogram!')
    A_fac_hist = r.TH1D('AWeightUncFacs', 'AWeightUncFacs',
            a_weight_hist.GetNbinsX(),
            a_weight_hist.GetBinLowEdge(1), 
            a_weight_hist.GetBinLowEdge(a_weight_hist.GetNbinsX()+1))
    
    for i_bin in range(1, a_weight_hist.GetNbinsX() + 1):
            new_err = a_weight_hist.GetBinError(i_bin) \
                * A_weight_unc_facs[i_bin - 1]

            a_weight_hist.SetBinError(i_bin, new_err)
    
            A_fac_hist.SetBinContent(i_bin, A_weight_unc_facs[i_bin - 1])
        
    c_fac = r.TCanvas()
    A_fac_hist.Draw()
    A_fac_hist.GetXaxis().SetRangeUser(30,100)
    A_fac_hist.GetYaxis().SetRangeUser(1,1.01)
    c_fac.Draw()
    
    before_r = a_weight_fit.GetParameter(4)
    
    c.cd()
    resids, fft = fit_and_fft(
        a_weight_hist, a_weight_fit, 'fullFitAWeightUncFacs',
        config['fit_options'],
        config['fit_start'], config['extended_fit_end'])
    
    after_r = a_weight_fit.GetParameter(4)
    
    c, _ = plot_hist(a_weight_hist)
    c.Draw()
    c2, _ = plot_fft(fft, cbo_freq)
    c2.Draw()
    
    print(f'Before: {before_r}')
    print(f'After: {after_r}')
    print(f'Diff: {after_r - before_r}')

## A-weighted Correlations

In [ ]:
%%time 
a_weight_fr = a_weight_hist.Fit(a_weight_fit, 'EMqS', '', config['fit_start'], config['extended_fit_end'])

In [ ]:
a_corr_mat = a_weight_fr.GetCorrelationMatrix()
for par_num in range(a_weight_fit.GetNpar()):
    if is_free_param(a_weight_fit, par_num):
        par_name = a_weight_fit.GetParName(par_num).replace('#', '\\')
        val = a_weight_fit.GetParameter(par_num)
        err = a_weight_fit.GetParError(par_num)
        corr =  a_corr_mat(4, par_num)
        display(Latex(f'${par_name}$: {val} $\pm$ {err}, R-corr {corr:.3f}'))

### A-Weighted Wrapped Plot and Residuals Hist

In [ ]:
c, _ = make_wrapped_wiggle_plot(a_weight_hist, a_weight_fit, 'A-Weighted', 
                               config['fit_start'], config['extended_fit_end'])
c.Draw()
r.gStyle.SetOptFit(0)

c.Print('aWeightWrapped.pdf')

In [ ]:
early_resids = build_residuals_hist(a_weight_hist, a_weight_fit, use_errors=True,
                                   name='AWeightedResidsErrorWeighted')

r.gStyle.SetOptTitle(0)

c = r.TCanvas()
c.Divide(1,2,0,0)
c.cd(1)
resid_A_hist = a_weight_hist.Clone()
resid_A_hist.SetName('residAHist')
resid_A_hist.Draw()
resid_A_hist.GetXaxis().SetRangeUser(config['fit_start']-20, config['fit_start']+40)

c.cd(2)
early_resids.Draw()
early_resids.SetTitle('A-Weighted Residuals; time [#mus]; pull')
early_resids.GetYaxis().SetRangeUser(-5, 5)
early_resids.GetXaxis().SetRangeUser(config['fit_start']-20, config['fit_start']+40)
early_resids.Draw()
line = r.TLine(config['fit_start'], -5, config['fit_start'], 5)
line2 = r.TLine(config['fit_start']-20, 0, config['fit_start']+40, 0)
line.Draw()
line2.Draw()
c.Draw()

In [ ]:
r.gStyle.SetOptTitle(1)
r.gStyle.SetOptFit(1111)

## A-Weighted Stop Time Scan

In [ ]:
stop_time_fit = clone_full_fit_tf1(a_weight_fit, 'a_stop_time_fit')
#fix a lot of parameters for the start time scan
# for par_num in start_time_conf['params_to_fix']:
for par_num in range(5, 27):
    stop_time_fit.FixParameter(par_num, 
                                stop_time_fit.GetParameter(par_num))

In [ ]:
%%time
if config['do_start_time_scans']:
    a_stop_chi2g, stop_scan_res = stop_time_scan(a_weight_hist, stop_time_fit,
                                  start=config['fit_start'],
                                  end=config['extended_fit_end'],
                                  step=2, n_pts=150,
                                  fit_options=config['fit_options']+'E')

In [ ]:
if config['do_start_time_scans']:
    canvs = [r.TCanvas()]
    a_stop_chi2g.Draw('ap')
    canvs[-1].Draw()

    for i, res in enumerate(stop_scan_res):
        canvs.append(r.TCanvas())
        res.Draw()
        canvs[-1].Draw()

## A-Weighted Start Time Scan

In [ ]:
a_start_time_fit = clone_full_fit_tf1(a_weight_fit, 'a_start_time_fit')
# fix a lot of parameters for the start time scan
# for par_num in start_time_conf['params_to_fix']:
for par_num in range(5, 27):
    a_start_time_fit.FixParameter(par_num, 
                                  a_start_time_fit.GetParameter(par_num))

In [ ]:
%%time
if config['do_start_time_scans']:
    a_scan_chi2, scan_res = start_time_scan(a_weight_hist, a_start_time_fit,
                              start=config['fit_start'],
                              end=config['extended_fit_end'],
                              step=start_time_conf['step'], n_pts=start_time_conf['n_pts'],
                              fit_options=config['fit_options']+'E')

In [ ]:
if config['do_start_time_scans']:
    canvs = [r.TCanvas()]
    a_scan_chi2.Draw('ap')
    canvs[-1].Draw()

    for i, res in enumerate(scan_res):
        canvs.append(r.TCanvas())
        res.Draw()
        canvs[-1].Draw()

## A-Weighted Pileup Multiplier Scan

In [ ]:
a_pu_fit = clone_full_fit_tf1(a_weight_fit, 'a_pu_fit')

In [ ]:
%%time
scale_factors = [i/10 + 0.4 for i in range(10)]
a_pu_scan_fits = A_weight_pu_mult_scan(all_calo_2d, uncorrected_2d, 
                          a_vs_e_spline, a_pu_fit,
                          scale_factors, config)

In [ ]:
a_chi2_g, a_par_gs = make_pu_scan_graphs(scale_factors, a_pu_scan_fits)

In [ ]:
c1 = plot_pu_sweep_R(a_par_gs, 'A-Weighted')
c1.Draw()
c2 = plot_pu_sweep_chi2(a_chi2_g, 'A-Weighted')
c2.Draw()

### A weighted calo sweep

In [ ]:
%%time
per_calo_a_fit = clone_full_fit_tf1(a_weight_fit, 'per_calo_a_fit')

# free 2*omega_cbo params for per calo
# for par_num in range(24, 27):
#     per_calo_a_fit.ReleaseParameter(par_num)

# limit the cbo lifetime params
per_calo_a_fit.SetParLimits(6, 50, 400)
# per_calo_a_fit.SetParLimits(24, 30, 200)

# fix vw parameters for the single calo fit
for par_num in [10, 13]:
    per_calo_a_fit.FixParameter(par_num,
                                per_calo_a_fit.GetParameter(par_num))

# T-method fits per calo
calo_sweep_a_res = A_weighted_calo_sweep(
    master_3d, per_calo_a_fit, a_vs_e_spline, config)
calo_a_chi2_g, calo_sweep_a_par_gs = make_calo_sweep_graphs(
    calo_sweep_a_res)

In [ ]:
r.gStyle.SetOptFit(0)
r.gStyle.SetOptFit(1111)
r.gStyle.SetStatH(0.15)

canvs = []
lns = []

calo_a_chi2_g.SetName('calo_chi2_g')
calo_a_chi2_g.SetTitle(';calo num; #chi^{2}/ndf')

for hist, fit, _, fft in calo_sweep_a_res:
    
    cbo_freq = fit.GetParameter(9) / 2 / math.pi
    
    c, _ = plot_hist(hist)
    canvs.append(c)
    r.gStyle.SetOptFit(0)
    r.gStyle.SetOptFit(1111)
    hist.GetYaxis().SetRangeUser(1, hist.GetMaximum()*1.5)
    canvs[-1].Draw()

    c, new_lns = plot_fft(fft, cbo_freq)
    canvs.append(c)
    lns.extend(new_lns)
    canvs[-1].Draw()

In [ ]:
c = r.TCanvas()
calo_a_chi2_g.Draw('ap')
calo_a_chi2_g.GetXaxis().SetLimits(0, 25)
ln = r.TLine(0, 1, 25, 1)
ln.SetLineWidth(2)
ln.SetLineStyle(2)
ln.Draw()
c.Draw()

In [ ]:
canvs = []
for par_num in calo_sweep_a_par_gs:
    canvs.append(r.TCanvas())
    g = calo_sweep_a_par_gs[par_num]
    g.GetXaxis().SetTitle('calo num')

    g.Draw('ap')

    par_name = g.GetYaxis().GetTitle()
    par_name = strip_par_name(par_name)

    if par_name == 'R':
        g.Fit('pol0')
    
    g.GetXaxis().SetLimits(0, 25)

    canvs[-1].Draw()